In [1]:
import time
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import faker
import pandas as pd
from tqdm import tqdm
from datetime import datetime

In [2]:
fake = faker.Faker(locale='ru')

In [3]:
yandex_headers = {
    'User-Agent': fake.chrome(),
    'accept-language': 'en-US,en;q=0.9',
    'pragma': 'np-cache',
    'content-type': 'application/json',
    'referer': 'http://yandex.ru/jobs/vacancies'
}

In [4]:
API_URL = "http://yandex.ru/jobs/api/publications/?page_size=1000&text="
VACANCY_INFO_URL = "http://yandex.ru/jobs/vacancies/"

In [5]:
# proxy = {
#     'http': 'http://130.61.236.9:80'
# }

In [6]:
clear_string = lambda x: re.sub(' +', ' ', x.replace('\n', '\n ')).strip()
title_to_url = lambda x: "-".join(re.sub('\s+', ' ', re.sub(r'[^\w\s-]', '', x.lower())).split(' '))

In [7]:
result = requests.get(API_URL, headers=yandex_headers)
result

<Response [200]>

In [8]:
all_jobs = result.json()['results']
len(all_jobs)

854

In [9]:
datetime.now()

datetime.datetime(2022, 8, 10, 6, 46, 30, 182347)

In [10]:
jobs_info = []
for job in all_jobs:
    url = VACANCY_INFO_URL + title_to_url(job['title']) + '-' + str(job['id'])
    internal_id = job['id']
    summary = job['short_summary']
    jobs_info.append((url, internal_id, job['title']))

In [12]:
yandex_jobs = []
for job in tqdm(jobs_info):
    result = requests.get(job[0], headers=yandex_headers)

    if result.status_code != 200:
        print(f'Non 200 for {job[0]}. skipping...')
        continue

    soup = BeautifulSoup(result.text)

    description_div = soup.find('div', {'class': 'lc-jobs-vacancy__section lc-jobs-vacancy__section_type_description_main'})
    description = clear_string(description_div.text) if description_div is not None else None

    responsibilities_div = soup.find('div', {'class': 'lc-jobs-vacancy__section lc-jobs-vacancy__section_type_duties'})
    responsibilities = clear_string(responsibilities_div.text) if responsibilities_div is not None else None

    qualifications_div = soup.find('div', {'class': 'lc-jobs-vacancy__section lc-jobs-vacancy__section_type_key_qualifications'})
    qualifications = clear_string(qualifications_div.text) if qualifications_div is not None else None

    if description is None and responsibilities is None and qualifications is None:
        print(f'All None for {job[0]}. skipping...')
        continue

    job_dict = {
        'title': job[2],
        'internal_id': job[1],
        'url': job[0],
        'description': description,
        'responsibilities': responsibilities,
        'qualifications': qualifications,
        'company': 'Yandex',
        'publish_date': None
    }
    yandex_jobs.append(job_dict)

    time.sleep(0.2) # sleep for 200 ms

100%|██████████| 854/854 [15:24<00:00,  1.08s/it]


In [13]:
snapshot = pd.DataFrame(yandex_jobs)
snapshot.sample(5)

,title,internal_id,url,description,responsibilities,qualifications,company,publish_date
639,Технический менеджер проектов в группу разрабо...,9137,http://yandex.ru/jobs/vacancies/технический-ме...,Мы создаём инфраструктуру для переноса всей би...,Что нужно делать:\n помогать бизнес-заказчикам...,"Мы ждем, что вы:\n работали с распределёнными ...",Yandex,None
560,Юрист-методолог банковских процессов в Финтех,9287,http://yandex.ru/jobs/vacancies/юрист-методоло...,"Мы создаём платёжные продукты, сервисы и инстр...",Что нужно делать:\n готовить внутренние нормат...,"Мы ждем, что вы:\n имеете высшее юридическое о...",Yandex,None
98,Дата-инженер в Плюс,9393,http://yandex.ru/jobs/vacancies/дата-инженер-в...,Наша команда строит DWH для Медиасервисов Янде...,Что нужно делать:\n анализировать источники да...,"Мы ждем, что вы:\n занимались промышленной раз...",Yandex,None
778,Ночной инженер технической поддержки в Yandex....,7905,http://yandex.ru/jobs/vacancies/ночной-инженер...,Вашей основной задачей будет решение проблем к...,Что нужно делать:\n консультировать клиентов Y...,"Мы ждем, что вы:\n грамотно пишете и говорите ...",Yandex,None
371,Разработчик на Python в Яндекс ID,4433,http://yandex.ru/jobs/vacancies/разработчик-на...,"Система Яндекс ID — это гораздо больше, чем «д...",None,"Мы ждем, что вы:\n разрабатываете на Python не...",Yandex,None


In [14]:
snapshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 854 entries, 0 to 853
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             854 non-null    object
 1   internal_id       854 non-null    int64 
 2   url               854 non-null    object
 3   description       815 non-null    object
 4   responsibilities  800 non-null    object
 5   qualifications    854 non-null    object
 6   company           854 non-null    object
 7   publish_date      0 non-null      object
dtypes: int64(1), object(7)
memory usage: 53.5+ KB


In [15]:
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

'10-08-2022'

In [16]:
snapshot.to_csv(f'../data/yandex/{current_date}.csv')
snapshot.to_excel(f'../data/yandex/{current_date}.xlsx', engine='xlsxwriter')

#### Проверка на единичной вакансии

In [195]:
result = requests.get(jobs_info[19][0], headers=yandex_headers)
result

<Response [200]>

In [196]:
result.request.url

'https://yandex.ru/jobs/vacancies/python-%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA-%D0%B2-wiki-4005'

In [197]:
soup = BeautifulSoup(result.text)

In [198]:
clear_string(soup.find('div', {'class': 'lc-jobs-vacancy__section lc-jobs-vacancy__section_type_description_main'}).text)

'Наша команда делает сервисы, которые формируют базу знаний Яндекса. Ежедневно сотрудники Яндекса обращаются за рабочей документацией, совместно редактируют статьи, делятся рабочими материалами, готовят отчеты. Работоспособность всех этих процессов обеспечивается нашими решениями.\n Исторически большинство крупных проектов написано на Python3 (Django, FastApi).'

In [199]:
clear_string(soup.find('div', {'class': 'lc-jobs-vacancy__section lc-jobs-vacancy__section_type_duties'}).text)

'Что нужно делать:\n повышать удобство работы с сервисами;\n улучшать быстродействие;\n проектировать и внедрять функциональность в ответ на новые потребности пользователей и бизнеса.'

In [200]:
clear_string(soup.find('div', {'class': 'lc-jobs-vacancy__section lc-jobs-vacancy__section_type_key_qualifications'}).text)

'Мы ждем, что вы:\n имеете опыт разработки (production-ready) и поддержки веб-сервисов. При этом важен не столько сам язык, сколько умение мыслить, писать эффективный код и обладать собственным видением;\n любите сложные и крупные сервисы;\n готовы к тесному взаимодействию с потребителями.'